## Import package

In [1]:
# ! pip install factor_analyzer pingouin stargazer

In [2]:
# for data ETL
import pandas as pd
import numpy as np
import ETL
# for data EPA
import plotly.express as px
import seaborn as sns               
import matplotlib.pyplot as plt   
# for establish index
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import pingouin as pg # index reliability testing
# for stats ml
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.miscmodels.ordinal_model import OrderedModel
from stargazer.stargazer import Stargazer

## Load data and filiter 

In [3]:
raw_data = pd.read_csv('raw_data.csv')

In [4]:
filter_data = raw_data.loc[raw_data['目前國內政黨當中，請問您是否偏向哪一個政黨？'].isin(['沒有特定支持', '都不支持'])]
filter_data.drop(['您有絶對的權力決定是否要參與本研究。若您願意參與，請務必勾選下列選項：', '請填寫您的電子信箱，以利後續抽獎聯繫使用'], axis=1, inplace=True)

/var/folders/dn/wbyhv8k14697mm1zy19w1jh40000gn/T/ipykernel_20726/2097987034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data.drop(['您有絶對的權力決定是否要參與本研究。若您願意參與，請務必勾選下列選項：', '請填寫您的電子信箱，以利後續抽獎聯繫使用'], axis=1, inplace=True)


## Data Cleaning

In [5]:
ml_df = ETL.data_cleaning(filter_data)
ml_df = ml_df.reset_index(drop=True)

/Users/wen/Desktop/github/media_political/ETL.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data['TV_news_time'] = filter_data['請問您平均每天花多少時間注意電視上的選舉新聞？'].apply(ETL_function.news_time)
/Users/wen/Desktop/github/media_political/ETL.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data['news_paper_time'] = filter_data['請問您平均每天花多少時間注意報紙上的選舉新聞？'].apply(ETL_function.news_time)
/Users/wen/Desktop/github/media_political/ETL.py:16: SettingWithCopyWarning: 
A value is trying to be set on a co

In [6]:
ml_df = ETL.feature_engineering(ml_df)
ml_df = ETL.DV_feature(ml_df)
ml_df = ETL.anti_party_feature(ml_df)
ml_df.shape

(503, 51)

## Modleing

In [7]:
ml_df.columns

Index(['TV_news_time', 'news_paper_time', 'int_news_time', 'TV_debate',
       'read_media', 'like_media', 'share_media', 'comment_media',
       'int_discuss', 'read_election_news', 'read_election_leaflet',
       'convince', 'campaign', 'volunteer', 'election_mayor', 'election_18',
       'sex', 'ethnic', 'edu', 'income', 'pk_1', 'pk_2', 'pk_3',
       'political_knowledge', 'anti_1', 'anti_2', 'anti_3', 'anti_4', 'anti_5',
       'TC_issue', 'Negative_1', 'Negative_2', 'Negative_3', 'online_pp',
       'offline_pp', 'anti_party', 'online_media_pp_mean', 'voting_mean',
       'offline_media_pp_mean', 'campaign_worker_pp_mean',
       'online_media_pp_score', 'voting_score', 'offline_media_pp_score',
       'campaign_worker_pp_score', 'onlnie_scores', 'offlnie_scores',
       'political_polarization_mean', 'party_image_mean',
       'political_polarization_score', 'party_image_score',
       'anti_party_scores'],
      dtype='object')

### DV ~ anti_party

In [8]:
# Define the dependent and independent variables
dependent_variables = ['online_media_pp_mean', 'voting_mean', 'offline_media_pp_mean', 'campaign_worker_pp_mean']
independent_variables = ['sex', 'C(ethnic, Treatment(reference="臺灣人"))', 'edu', 'income', 'political_knowledge', 'TC_issue', 'political_polarization_mean', 'party_image_mean', 'C(Negative_1, Treatment(reference="沒有影響"))', 'C(Negative_2, Treatment(reference="沒有影響"))', 'C(Negative_3, Treatment(reference="沒有影響"))']

# Create the models and print the summaries
for dep_var in dependent_variables:
    formula = dep_var + ' ~ ' + ' + '.join(independent_variables)
    model = smf.ols(formula, data=ml_df).fit()
    print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     online_media_pp_mean   R-squared:                       0.076
Model:                              OLS   Adj. R-squared:                  0.035
Method:                   Least Squares   F-statistic:                     1.874
Date:                  Fri, 14 Jul 2023   Prob (F-statistic):             0.0111
Time:                          20:15:06   Log-Likelihood:                -312.35
No. Observations:                   503   AIC:                             668.7
Df Residuals:                       481   BIC:                             761.5
Df Model:                            21                                         
Covariance Type:              nonrobust                                         
                                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------